# Compare allele counts of in-house data to that of ALFA data using a Fisher's Exact Test of significance. Correct for multiple testing

In [1]:
# Change working directory

import os
os.chdir(r"C:\Users\User\Desktop\Megan\MSC2\Results\5._Posthoc_analysis\Pipeline_GnomAD_SAHGP_14032023\Genomic_data_analysis\Analysis")

In [2]:
# Import modules and packages

import itertools
import os
import numpy as np
import pandas as pd
import json
import seaborn as sns
from matplotlib import pyplot as plt
from upsetplot import plot
from requests import get, codes as http_code
from warnings import simplefilter
import sys
sys.path.append(r"C:\Users\User\Desktop\Megan\MSC2\Results\5._Posthoc_analysis\Pipeline_GnomAD_SAHGP_14032023\Genomic_data_analysis")
import Utils.constants as constants
import Utils.functions as functions

In [66]:
# Import merged in-house and ALFA data. This data is only on variants with rsIDs.

ih_alfa_data = pd.read_csv(
    os.path.join(
        constants.HOME_PATH,
        "Data",
        "Processed",
        "IH_ALFA_allele_counts.csv",
    )
).drop(columns="Unnamed: 0")

ih_alfa_data.head(5)

# Filter out data with the exception of ALFA African Others, EUR, EAS, SAS, and averaged Recent African counts
ih_alfa_data = ih_alfa_data.rename(columns={"REF": "REF_AL", "ALT": "ALT_AL"})
ih_alfa_data = ih_alfa_data.loc[:, (ih_alfa_data.columns.str.contains("GENE") | ih_alfa_data.columns.str.contains("POS") | ih_alfa_data.columns.str.contains("ALT_AL") | ih_alfa_data.columns.str.contains("REF_AL") | ih_alfa_data.columns.str.contains("ID") | ih_alfa_data.columns.str.contains("ALFA_African Others") | ih_alfa_data.columns.str.contains("ALFA_European") | ih_alfa_data.columns.str.contains("ALFA_East Asian") | ih_alfa_data.columns.str.contains("ALFA_South Asian") | ih_alfa_data.columns.str.contains("Recent African"))]
ih_alfa_data = ih_alfa_data.rename(columns={"REF_AL": "REF", "ALT_AL": "ALT"})
ih_alfa_data.head(5)

,ID,REF,ALT,GENE,POS,ALT_CT_IH_Recent African,REF_CT_IH_Recent African,CORR_REF_CT_IH_Recent African,ALT_CT_ALFA_African Others,ALT_CT_ALFA_East Asian,ALT_CT_ALFA_European,ALT_CT_ALFA_South Asian,REF_CT_ALFA_African Others,REF_CT_ALFA_East Asian,REF_CT_ALFA_European,REF_CT_ALFA_South Asian
0,rs552586867,C,G,COL4A1,110148891,2,1606,1654,0.0,0.0,0.0,0.0,114.0,86.0,9690.0,98.0
1,rs59409892,C,G,COL4A1,110148917,153,1461,1503,9.0,0.0,0.0,0.0,105.0,86.0,9824.0,98.0
2,rs535182970,G,C,COL4A1,110148920,0,1608,1656,0.0,0.0,0.0,0.0,114.0,86.0,9690.0,98.0
3,rs56406633,A,G,COL4A1,110148959,0,1608,1656,0.0,3.0,444.0,3.0,114.0,83.0,13842.0,95.0
4,rs568536001,G,C,COL4A1,110148971,0,1608,1656,0.0,0.0,0.0,0.0,114.0,86.0,9690.0,98.0


In [67]:
# Remove rows where population groups alt allele counts are 0. These rows contain variants that are not present in one of the populations.

ih_alfa_majorpop_union = ih_alfa_data.replace(0,np.NAN).dropna(subset=ih_alfa_data.loc[:, ih_alfa_data.columns.str.contains("ALT_CT")].columns)

ih_alfa_majorpop_union = ih_alfa_majorpop_union.replace(np.NAN, 0)
ih_alfa_majorpop_union.head(5)

,ID,REF,ALT,GENE,POS,ALT_CT_IH_Recent African,REF_CT_IH_Recent African,CORR_REF_CT_IH_Recent African,ALT_CT_ALFA_African Others,ALT_CT_ALFA_East Asian,ALT_CT_ALFA_European,ALT_CT_ALFA_South Asian,REF_CT_ALFA_African Others,REF_CT_ALFA_East Asian,REF_CT_ALFA_European,REF_CT_ALFA_South Asian
29,rs13260,G,T,COL4A1,110149776,435.0,1199.0,1221.0,33.0,2.0,11567.0,7.0,163.0,496.0,115251.0,169.0
113,rs75273185,C,T,COL4A1,110152180,4.0,1604.0,1652.0,1.0,3.0,446.0,3.0,113.0,83.0,13840.0,95.0
139,rs681884,G,A,COL4A1,110152715,1590.0,64.0,66.0,194.0,101.0,16697.0,99.0,14.0,1.0,8087.0,17.0
181,rs78326356,G,A,COL4A1,110153931,15.0,1593.0,1641.0,1.0,6.0,438.0,3.0,113.0,114.0,13926.0,101.0
189,rs1192201,G,A,COL4A1,110154143,1553.0,103.0,103.0,243.0,428.0,159198.0,4665.0,21.0,126.0,14128.0,377.0


In [68]:
# Generate all possible combinations of ALFA and in-house populations for comparison

comp_populations = [
    # "IH_SA",
    # "IH_WA",
    # "IH_ASW",
    # "IH_EA",
    # "IH_CA",
    # "IH_ACB",
]
combinations = list(itertools.combinations(comp_populations, 2))
combinations.extend([("IH_Recent African", "ALFA_East Asian"), ("IH_Recent African", "ALFA_South Asian"), ("IH_Recent African", "ALFA_European"), ("IH_Recent African", "ALFA_African Others")])


In [69]:
# We would like to perform the Fishers test on corrected in-house reference counts only. To make the analysis easier, we will drop columns with non-corrected in-house reference counts. Corrected reference counts will be renamed to reference counts for the sake of brevity.

ih_alfa_ref_drop = ih_alfa_majorpop_union.loc[:,~ih_alfa_majorpop_union.columns.str.startswith('REF_CT_IH')]

ih_alfa_ref_drop.columns = ih_alfa_ref_drop.columns.str.replace('CORR_','')

In [70]:
ih_alfa_ref_drop

,ID,REF,ALT,GENE,POS,ALT_CT_IH_Recent African,REF_CT_IH_Recent African,ALT_CT_ALFA_African Others,ALT_CT_ALFA_East Asian,ALT_CT_ALFA_European,ALT_CT_ALFA_South Asian,REF_CT_ALFA_African Others,REF_CT_ALFA_East Asian,REF_CT_ALFA_European,REF_CT_ALFA_South Asian
29,rs13260,G,T,COL4A1,110149776,435.0,1221.0,33.0,2.0,11567.0,7.0,163.0,496.0,115251.0,169.0
113,rs75273185,C,T,COL4A1,110152180,4.0,1652.0,1.0,3.0,446.0,3.0,113.0,83.0,13840.0,95.0
139,rs681884,G,A,COL4A1,110152715,1590.0,66.0,194.0,101.0,16697.0,99.0,14.0,1.0,8087.0,17.0
181,rs78326356,G,A,COL4A1,110153931,15.0,1641.0,1.0,6.0,438.0,3.0,113.0,114.0,13926.0,101.0
189,rs1192201,G,A,COL4A1,110154143,1553.0,103.0,243.0,428.0,159198.0,4665.0,21.0,126.0,14128.0,377.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14894,rs73061248,C,A,CARD8,48255857,223.0,1433.0,14.0,31.0,2705.0,39.0,100.0,57.0,11581.0,59.0
14897,rs12984829,T,G,CARD8,48255879,244.0,1412.0,39.0,1.0,2648.0,2.0,287.0,497.0,109676.0,184.0
14900,rs12984859,T,C,CARD8,48255916,467.0,1189.0,27.0,31.0,2959.0,40.0,87.0,57.0,11327.0,58.0
14902,rs62129827,C,T,CARD8,48255931,223.0,1433.0,14.0,30.0,2704.0,39.0,100.0,56.0,11582.0,59.0


In [84]:
# Calculate fisher's exact odds ratios and p-values for each variant for the different population combination

fishers_results = functions.fishers_test(ih_alfa_ref_drop, combinations)
fishers_results.head(5)
fishers_results.groupby("GENE").nunique()

,ID,REF,ALT,POS,PVALUE_IH_Recent African_ALFA_East Asian,OR_IH_Recent African_ALFA_East Asian,PVALUE_IH_Recent African_ALFA_South Asian,OR_IH_Recent African_ALFA_South Asian,PVALUE_IH_Recent African_ALFA_European,OR_IH_Recent African_ALFA_European,PVALUE_IH_Recent African_ALFA_African Others,OR_IH_Recent African_ALFA_African Others
GENE,,,,,,,,,,,,
AGT,206,4,4,206,178,186,186,186,182,200,178,190
AP4B1,14,4,4,14,12,14,12,12,13,14,11,13
CARD8,229,4,4,229,199,201,201,199,222,226,189,205
CAT,67,4,4,67,66,66,65,65,67,67,56,64
COL4A1,623,4,4,623,565,534,563,542,595,602,523,584
IL10,13,4,4,13,13,13,13,13,13,13,12,13
IL1B,14,4,3,14,14,13,14,14,14,14,12,14
IL6,17,4,4,17,15,14,17,17,13,17,15,16
MTHFR,59,4,4,59,53,47,54,45,51,59,56,58


In [72]:
# Save Fisher's test results to CSV file

fishers_results.reset_index(drop=True).to_csv(
    os.path.join(
        constants.HOME_PATH,
        "Results",
        "Stats",
        "Fishers_results_subpopulations.csv",
    )
)


In [73]:
# Correct for multiple testing using the Bonferroni method

# Generate a list with column names for all subpopulation p-values in the fishers_results dataframe
p_value_combinations_list = []
for combination in combinations:
    first_pop = combination[0]
    second_pop = combination[1]
    p_value_combinations = "PVALUE_{}_{}".format(first_pop, second_pop)
    p_value_combinations_list.append(p_value_combinations)

# # Method 1: Correct p-values for multiple testing within each subpopulation
# multipletests_results = functions.multipletest_correction_percolumn(
#     fishers_results, p_value_combinations_list, 0.05, "bonferroni"
# )

# multipletests_results.to_csv(
#     os.path.join(
#         constants.HOME_PATH,
#         "Results",
#         "Stats",
#         "Fishers_multipletestcorrection_bonf_pd.csv",
#     )
# )

# Method 2: Correct for multiple testing for the entire dataset without segregating the data per subpopulation
multipletests_results = functions.multipletest_correction_wholedf(
    fishers_results, p_value_combinations_list, 0.05, "bonferroni"
)

multipletests_results.to_csv(
    os.path.join(
        constants.HOME_PATH,
        "Results",
        "Stats",
        "Fishers_multipletestcorrection_bonf_wd.csv",
    )
)


## Combine corrected p-values, odds ratios, allele counts and alt allele frequencies into single dataframe

In [74]:
# Format corrected p-value results in suitable format for combination with the other results

pvalue_table = multipletests_results.melt(id_vars=["ID", "REF", "ALT"]).rename(columns={"variable":"COMP_POPS", "value":"CORR_PVALUE"})
pvalue_table["COMP_POPS"] = pvalue_table["COMP_POPS"].apply(lambda x: str(x).replace('PVALUE_', ''))
pvalue_table.head(5)

,ID,REF,ALT,COMP_POPS,CORR_PVALUE
0,rs1000989,T,C,IH_Recent African_ALFA_African Others,1.0
1,rs1000990,T,C,IH_Recent African_ALFA_African Others,1.0
2,rs1005573,C,T,IH_Recent African_ALFA_African Others,1.0
3,rs1007311,A,G,IH_Recent African_ALFA_African Others,1.0
4,rs1008140,T,C,IH_Recent African_ALFA_African Others,1.0


In [75]:
# Format odds ratio results in suitable format for combination with the other results

fishers_df_OR_columns = fishers_results.columns.str.contains('OR_*|ID|REF|ALT')
or_table = fishers_results.iloc[:,fishers_df_OR_columns].melt(id_vars=["ID", "REF", "ALT"]).rename(columns={"variable":"COMP_POPS", "value":"OR"})
or_table["COMP_POPS"] = or_table["COMP_POPS"].apply(lambda x: str(x).replace('OR_', ''))
or_table.head(5)

,ID,REF,ALT,COMP_POPS,OR
0,rs13260,G,T,IH_Recent African_ALFA_East Asian,88.353808
1,rs75273185,C,T,IH_Recent African_ALFA_East Asian,0.066990
2,rs681884,G,A,IH_Recent African_ALFA_East Asian,0.238524
3,rs78326356,G,A,IH_Recent African_ALFA_East Asian,0.173675
4,rs1192201,G,A,IH_Recent African_ALFA_East Asian,4.438753


In [76]:
# Combine corrected p-values and odds ratios

pvalue_or_table = pd.merge(pvalue_table, or_table, on=["ID","REF","ALT","COMP_POPS"])
pvalue_or_table.head(5)

,ID,REF,ALT,COMP_POPS,CORR_PVALUE,OR
0,rs1000989,T,C,IH_Recent African_ALFA_African Others,1.0,1.020127
1,rs1000990,T,C,IH_Recent African_ALFA_African Others,1.0,0.950900
2,rs1005573,C,T,IH_Recent African_ALFA_African Others,1.0,0.940748
3,rs1007311,A,G,IH_Recent African_ALFA_African Others,1.0,1.036212
4,rs1008140,T,C,IH_Recent African_ALFA_African Others,1.0,1.169760


In [77]:
# Format allele count results in suitable format for combination with the other results

# Rename certain columns to prevent downstream problems when subsetting data
ih_alfa_data = ih_alfa_data.rename(columns= {'REF':'REF_AL', 'ALT':'ALT_AL'})

# Subset alt count data and format
ih_ALFA_data_ALT_columns = ih_alfa_data.columns.str.contains('ALT_CT_*|ID|REF_AL|ALT_AL')
alt_count_table = ih_alfa_data.iloc[:,ih_ALFA_data_ALT_columns].melt(id_vars=["ID", "REF_AL", "ALT_AL"], value_name="ALT_CTS", var_name="REG")

# Remove unnecessary info from comparison column
alt_count_table["REG"] = alt_count_table["REG"].apply(lambda x: str(x).replace('ALT_CT_', ''))

# Subset ref count data and format
ih_ALFA_data_REF_columns = ih_alfa_data.columns.str.contains('REF_CT_*|ID|REF_AL|ALT_AL')
ref_count_table = ih_alfa_data.iloc[:,ih_ALFA_data_REF_columns].melt(id_vars=["ID", "REF_AL", "ALT_AL"], value_name="REF_CTS", var_name="REG")

# Remove unnecessary info from comparison column
ref_count_table["REG"] = alt_count_table["REG"].apply(lambda x: str(x).replace('REF_CT_', ''))

# Combine formatted alt and ref count data
count_table = pd.merge(alt_count_table, ref_count_table, on=["ID", "REF_AL", "ALT_AL", "REG"]).rename(columns={'REF_AL':'REF', 'ALT_AL':'ALT'})
count_table.head(5)

,ID,REF,ALT,REG,ALT_CTS,REF_CTS
0,rs552586867,C,G,IH_Recent African,2.0,1606.0
1,rs59409892,C,G,IH_Recent African,153.0,1461.0
2,rs535182970,G,C,IH_Recent African,0.0,1608.0
3,rs56406633,A,G,IH_Recent African,0.0,1608.0
4,rs568536001,G,C,IH_Recent African,0.0,1608.0


In [78]:
# Combine significant p-value, odds ratio and allele count data. Calculate alt allele frequencies. 

# Filter p-value and odds ratio table by significant p-values

sign_pvalue_or_table = pvalue_or_table[pvalue_or_table.CORR_PVALUE < 0.05]

# Add empty count columns to p-value and odds ratio table
sign_pvalue_or_table["POP1_REF_CTS"] = ""
sign_pvalue_or_table["POP1_ALT_CTS"] = ""
sign_pvalue_or_table["POP2_REF_CTS"] = ""
sign_pvalue_or_table["POP2_ALT_CTS"] = ""

# Iterate over row in p-value and odds ratio table and append count data
for index, row in sign_pvalue_or_table.iterrows():
    
    # Extract variant ID, ref allele and alt allele information
    id = row['ID']
    ref = row['REF']
    alt = row['ALT']

    # Split comparison population column into the two comparison populations
    comp_pops = row['COMP_POPS']
    split_pops = comp_pops.split("_", 2)
    split_pops = ["_".join(split_pops[0:2])] + split_pops[2:]
    comp_pop1 = split_pops[0]
    comp_pop2 = split_pops[1]

    # Fetch row in count table with information for each comparison population 
    pop1_count_table_row = count_table.loc[(count_table.ID == id) & (count_table.REF == ref) & (count_table.ALT == alt) & (count_table.REG == comp_pop1)]
    pop2_count_table_row = count_table.loc[(count_table.ID == id) & (count_table.REF == ref) & (count_table.ALT == alt) & (count_table.REG == comp_pop2)]
    
    # Extract ref and alt count information for relevant comparison population
    pop1_ref = pop1_count_table_row['REF_CTS'].values
    pop1_alt = pop1_count_table_row['ALT_CTS'].values
    pop2_ref = pop2_count_table_row['REF_CTS'].values
    pop2_alt = pop2_count_table_row['ALT_CTS'].values

    # Append ref and alt count information to p-value and odds ratio table
    sign_pvalue_or_table.at[index, "POP1_REF_CTS"] = pop1_ref
    sign_pvalue_or_table.at[index, "POP1_ALT_CTS"] = pop1_alt
    sign_pvalue_or_table.at[index, "POP2_REF_CTS"] = pop2_ref
    sign_pvalue_or_table.at[index, "POP2_ALT_CTS"] = pop2_alt


# # Change count data format from list to float
sign_pvalue_or_table['POP1_REF_CTS'] = sign_pvalue_or_table['POP1_REF_CTS'].str[0]
sign_pvalue_or_table['POP1_ALT_CTS'] = sign_pvalue_or_table['POP1_ALT_CTS'].str[0]
sign_pvalue_or_table['POP2_REF_CTS'] = sign_pvalue_or_table['POP2_REF_CTS'].str[0]
sign_pvalue_or_table['POP2_ALT_CTS'] = sign_pvalue_or_table['POP2_ALT_CTS'].str[0]

# Calculate alt allele frequencies and append results to table
sign_pvalue_or_table["POP1_ALT_FREQ"] = sign_pvalue_or_table["POP1_ALT_CTS"].astype(float)/(sign_pvalue_or_table["POP1_ALT_CTS"].astype(float) + sign_pvalue_or_table["POP1_REF_CTS"].astype(float))
sign_pvalue_or_table["POP2_ALT_FREQ"] = sign_pvalue_or_table["POP2_ALT_CTS"].astype(float)/(sign_pvalue_or_table["POP2_ALT_CTS"].astype(float) + sign_pvalue_or_table["POP2_REF_CTS"].astype(float))

sign_pvalue_or_table.head(5)

C:\Users\User\AppData\Local\Temp\ipykernel_10908\1441399797.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_pvalue_or_table["POP1_REF_CTS"] = ""
C:\Users\User\AppData\Local\Temp\ipykernel_10908\1441399797.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_pvalue_or_table["POP1_ALT_CTS"] = ""
C:\Users\User\AppData\Local\Temp\ipykernel_10908\1441399797.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,ID,REF,ALT,COMP_POPS,CORR_PVALUE,OR,POP1_REF_CTS,POP1_ALT_CTS,POP2_REF_CTS,POP2_ALT_CTS,POP1_ALT_FREQ,POP2_ALT_FREQ
51,rs10851240,T,G,IH_Recent African_ALFA_African Others,4.463217e-28,0.006918,696.0,4.0,1652.0,28.0,0.005714,0.016667
84,rs11122582,C,T,IH_Recent African_ALFA_African Others,1.969724e-06,34.414226,940.0,700.0,956.0,1.0,0.426829,0.001045
94,rs112260270,G,A,IH_Recent African_ALFA_African Others,1.753378e-40,0.001260,131.0,1.0,1655.0,35.0,0.007576,0.020710
98,rs112790627,G,A,IH_Recent African_ALFA_African Others,1.539928e-25,0.007636,432.0,4.0,1652.0,26.0,0.009174,0.015495
132,rs118114021,A,T,IH_Recent African_ALFA_African Others,3.266652e-11,0.042657,23.0,15.0,1641.0,18.0,0.394737,0.010850


In [83]:
# Add gene and positional information

sign_pvalue_or_table = pd.merge(sign_pvalue_or_table, ih_alfa_data[["ID", "REF_AL", "ALT_AL", "GENE", "POS"]], how="left", left_on=["ID", "REF", "ALT"], right_on=["ID", "REF_AL", "ALT_AL"])

sign_pvalue_or_table.groupby("GENE").nunique()

,ID,REF,ALT,COMP_POPS,CORR_PVALUE,OR,POP1_REF_CTS,POP1_ALT_CTS,POP2_REF_CTS,POP2_ALT_CTS,...,ALT_AL_x,GENE_x,POS_x,REF_AL_y,ALT_AL_y,GENE_y,POS_y,REF_AL,ALT_AL,POS
GENE,,,,,,,,,,,,,,,,,,,,,
AGT,198,4,4,4,291,310,164,149,207,260,...,4,1,198,4,4,1,198,4,4,198
AP4B1,14,4,4,3,33,37,12,10,38,37,...,4,1,14,4,4,1,14,4,4,14
CARD8,221,4,4,4,397,402,153,141,257,294,...,4,1,221,4,4,1,221,4,4,221
CAT,60,4,4,4,124,124,45,43,105,111,...,4,1,60,4,4,1,60,4,4,60
COL4A1,519,4,4,4,942,918,394,392,461,650,...,4,1,519,4,4,1,519,4,4,519
IL10,13,4,4,3,26,26,12,10,25,23,...,4,1,13,4,4,1,13,4,4,13
IL1B,14,4,3,3,22,22,13,13,21,22,...,3,1,14,4,3,1,14,4,3,14
IL6,17,4,4,3,31,35,14,13,30,31,...,4,1,17,4,4,1,17,4,4,17
MTHFR,42,4,4,4,84,81,38,37,78,86,...,4,1,42,4,4,1,42,4,4,42


In [81]:
# Save results to table

sign_pvalue_or_table.to_csv(
    os.path.join(
        constants.HOME_PATH,
        "Results",
        "Stats",
        "Significant_corrected_Fishers_results.csv",
    )
)